# 부록 1. ChatOllama + create_react_agent

- [강의 질문: create_react_agent, retriever요약 문제점](https://www.inflearn.com/community/questions/1609482) 디버깅입니다
- [create_retriever_tool](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.retriever.create_retriever_tool.html#create-retriever-tool) + [create_react_agent](https://python.langchain.com/api_reference/langchain/agents/langchain.agents.react.agent.create_react_agent.html)의 조합이 사용자의 질문에 답변을 생성하지 않고, 문서를 요약하는 케이스를 디버깅합니다

In [3]:
import os

from langchain_chroma import Chroma
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.tools.retriever import create_retriever_tool

embedding_function = AzureOpenAIEmbeddings(
    model='text-embedding-3-large',
    azure_endpoint=os.getenv('AZURE_OPENAI_EMBEDDING_ENDPOINT'),
    api_key=os.getenv('AZURE_OPENAI_EMBEDDING_API_KEY')
)
vector_store = Chroma(
    embedding_function=embedding_function,
    collection_name = 'real_estate_tax',
    persist_directory='./real_estate_tax_collection'
)
retriever = vector_store.as_retriever(search_kwargs={'k': 3})
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name='real_estate_tax_retriever',
    description='Contains information about real estate tax up to December 2024',
)

## ChatOllama 활용방법

- 먼저 필요한 패키지를 설치합니다 (`uv sync`를 하셨다면 자동으로 설치)
    - `langchain-ollama`, `ollama` 설치 필요
- `ollama pull <모델이름>` 을 통해 활용할 모델을 다운로드 후 사용 가능

In [4]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0,
    # extract_reasoning = True
)  

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

tools = [retriever_tool]

graph = create_react_agent(
            model = llm,
            tools = tools,
            # 최근 버전에서는 `state_modifier`가 `prompt`로 변경됨
            state_modifier=("""당신은 문서 담당자자 입니다. 주어진 질문에 답변하세요. 의견이 아닌 사실만 제공하세요
                    문서 관련련 질문은 retriever_tool 도구를 사용하세요.
                    반환된 문서의 내용을 요약하지 말고 질문에 직접 답변하세요.
                    문서의 내용이 충분하지 않으면 '정보 부족'이라고 답하고, 확신 없는 내용을 만들어내지 마세요."""),
             )



In [7]:
inputs = {'messages': [HumanMessage("종합부동산세를 계산하는 방법을 알려주세요")]}
graph.invoke(inputs)

{'messages': [HumanMessage(content='종합부동산세를 계산하는 방법을 알려주세요', additional_kwargs={}, response_metadata={}, id='e3b09a19-1eeb-4fd5-92e3-638128e838e0'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2025-06-06T07:56:30.640786Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9009694500, 'load_duration': 22084458, 'prompt_eval_count': 275, 'prompt_eval_duration': 7313094041, 'eval_count': 30, 'eval_duration': 1672908709, 'model_name': 'llama3.1:latest'}, id='run-f98f8a53-2d98-4b95-8ea0-cd2963646f8b-0', tool_calls=[{'name': 'real_estate_tax_retriever', 'args': {'query': '종합부동산세 계산 방법'}, 'id': '9d313eab-13ae-4673-8e33-95c0bef604c2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 275, 'output_tokens': 30, 'total_tokens': 305}),
  ToolMessage(content='납세의무자가 2주택 이하를 소유한 경우\n\n| 과세표준              | 세율                                  |\n|---------------------|-------------------------------------|\n| 3억 원 이하           